In [0]:
import torch
import torch.nn as nn
from torch import *
from torch.nn.modules.conv import _ConvNd
from IPython.display import Math, HTML


Документация
https://pytorch.org/docs/stable/nn.html

## Convolutional neural network
![alt text](https://i.ibb.co/64h26nG/cnn-example.png)

# Операция свертки (conv)
## Начальное состояние, фильтр
![alt text](https://victorzhou.com/media/cnn-post/convolve-example-2.svg)
## Применим формулу свертки (пример для одного канала)

### $\sum\limits_{n,m=0..KS-1} w_{n,m}*x_{n+s,m+s}$
### ${n+s \leq H-1}, {m+s \leq W-1}$

# Результат

![alt text](https://victorzhou.com/media/cnn-post/convolve-output-1.svg)

## Замечание
При реализации свёртки просто матрица на вектор умножается. Например, реализация в cudnn Winograd transformation —https://arxiv.org/pdf/1509.09308.pdf

## Операция свертки с ядром 3х3х3 где 3х3 Weight*Height и 3-Channel numbers
![alt text](https://i.stack.imgur.com/CQtHP.gif)

## Output size calculation

* **F**ilter size = 3x3 -> 3
* **I**nput size = 5x5 -> 5
* **S**tride = 1x1 -> 1
* **P**adding = 0x0 -> 0

Output size = (I - F + 2*P)/S + 1 = (5 - 3 + 2*0)/1 + 1 = 3

Output size = 3 -> 3x3

![](https://miro.medium.com/max/1600/0*TsOwf6kzkUV8LZBX)

[About transposed convolutions](https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8)

# Инициализация весов фильтра и bias
[Техники эволюционируют:](https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78) zero, random, He, Xavier, ...

Выбор подходящей зависит от функций активации и архитектуры сети, [paper про RELU](https://arxiv.org/pdf/1704.08863.pdf).

## Инициализация фильтра и bias
## ${k}={1 \over C_{in}\prod\limits^1_{i=0}kernelsize[i]}$
Uniform distribution — равновероятностное появление значений в интервале от а до b, в нашем примере инициализации границы от $ -\sqrt{k}$ до $\sqrt{k}$

![alt text](http://openturns.github.io/openturns/1.9/_images/openturns-Uniform-1.png)

In [0]:
class Conv2d(_ConvNd):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1,
                 bias=True, padding_mode='zeros'):
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        super(Conv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            False, _pair(0), groups, bias, padding_mode)

    def conv2d_forward(self, input, weight):
        return F.conv2d(input, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input):
        return self.conv2d_forward(input, self.weight)

## Параметры
in_channels — количество каналов (на старте 3 - R,G,B)

out_channels — количество feature matrix (результат) на выходе, при этом каждый из результатов достигается фильтром kernel_size*in_channels

stride — параметр сдвига фильтра, так как мы используем 2dConvlayer направлений всего 2,стандартное значение 1

padding — на анимации выше
   
dilation — прореживание, при значении 1 не происходит

groups — разделение изображения на n-частей и применение набора фильтров для каждой части в отдельности с последующим конкатенированием до единой feature matrix

Метод _pair гененирит turple-кортеж, при инициализации параметров скаляром(x) возвращает обьект повторенный 2 раза — (x,x) (пример: in:kernel_size=1 out: kernel_size(1,1)), при явной инициализации(x,x) возвращает (x,x)

In [0]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


In [0]:
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

## ResNet - Kaiming He, Xiangyu Zhang ,Shaoqing Ren ,Jian Sun,(Microsoft Research,2015)
link:https://arxiv.org/pdf/1512.03385.pdf
![alt text](https://missinglink.ai/wp-content/uploads/2019/04/image1.png)
## Network structure 
![alt text](https://www.researchgate.net/profile/Seunghyoung_Ryu/publication/329954455/figure/fig1/AS:725290594623488@1549934161033/The-structure-of-ResNet-12.png)

## Basic block and Bottleneck block
![alt text](https://miro.medium.com/max/1980/1*j_lC2gsO1Kbia8PIQGHUZg.png)
![alt text](https://sun9-39.userapi.com/c858532/v858532493/1bb7/gKfLFz5MJXo.jpg)
![alt text](https://miro.medium.com/max/800/1*FqmD91PvbH7NKCnQWFJxvg.png)


## Основные проблемы сетей, предшествующих ResNet
### 1.Рост количества слоев не давал необходимого улучшения. 
### 2.Vanishing Gradient problem 
![alt text](https://miro.medium.com/max/2192/1*6A3A_rt4YmumHusvTvVTxw.png)
Так как в процессе backprop мы считаем произведения частных производных по параметрам с конца до начала сети, для сетей с большим количеством слоев мы сталкиваемся с проблемой очень низкого изменения параметров на последних слоях.

Картинка выше демонстрирует что чем больше наш выход от сигмоидной активации, тем производная ближе он к нулю. В результате мы получаем слишком маленькое значение градиента, а стандартный градиентный шаг 1e^-3:1e^-5 делает значение градиентного шага критически малым для изменения параметров.

## Придуманные решения
### 1.ReLU 

### 2.BatchNorm 

### 3.Resudial connections
Resudial connections основаны на идее прибавления input к результату прямого прохода через блок с дальнешим применением функции активации

Мы прибавляем к основному выходу от функции активации — F(x)=ReLU(W*x+b) так называемую identity function где F(x)=x, тем самым получаем F(x)+x

Основное преимущество данного подхода — если мы сталкиваемся с проблемой vanishing gradient или же мы уже получили достаточно хороший результат на предыдущем слое, то в таком случае приравняв F(x)=0 при активации ReLU мы будем получать одинаковый результат с n до k слоя так как ReLU = max(0,x) 

Пример вычислений приведен ниже
![alt text](https://miro.medium.com/max/960/1*0n8hSERRje182FEY61rPHQ.jpeg)


In [0]:
class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

Downsample — в случае когда мы имеем на входе матрицу 64х64 (обозначим x), а на выходе F(x) - 32x32 для подсчета ReLU(F(x)+x) нам необходимо соблюсти shape матрицы. В этом случае мы используем downsample

In [0]:
class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# Class «из ResNet»
Данный класс отнаследован, можно ещё через метод _make_layer

В параметры данного метода мы должны передать блок (Basic, Bottleneck, Customized), а так же параметры модели.

In [0]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# Average Pooling,Max Pooling
![alt text](https://www.researchgate.net/profile/Hanli_Wang2/publication/300020038/figure/fig2/AS:404961465782275@1473561745471/Toy-example-illustrating-the-drawbacks-of-max-pooling-and-average-pooling.png)

In [0]:
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

Если предполагается использовать уже готовую предтренированную архитектуру необходимо:


```
import torch
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
```






# ResNeXt
Saining Xie, Ross Girshick, Piotr Dollár, Zhuowen Tu, Kaiming He (2016)

У истоков данной модели стоят наработки Google и их Inception. После выхода ResNet, ребята из Google тоже решили протестировать данный принцип построения блоков и придумали InceptionResNet block-(b)
![alt text](https://miro.medium.com/max/1335/1*cIm3uy7eNvEchxRbBeBScQ.png)
Похожую идею паралельной многоступенчатой свертки применили и создатели ResNeXt-(a)

# Squeeze-and-Excitation ResNet
Jie Hu ,Li Shen,Samuel Albanie,Gang Sun,Enhua Wu (2017)
Данная архитектура обусловлена более сложным блоком 
![alt text](https://miro.medium.com/max/598/1*WNk-atKDUsZPvMddvYL01g.png)